In [21]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta
from selenium.webdriver.chrome.options import Options
import pandas as pd

# Function to convert the betting odds to integers while handling the signs
def convert_to_int(value):
    if value == 'EVEN':
        return 0
    if value.startswith('+'):
        return int(value[1:])
    elif value.startswith('-'):
        return int(value)
    else:
        return int(value)
    
def concat_values(x, y, z):
    return f"{x} {y} {z}"

def get_data():
    # Configure ChromeOptions for headless browsing
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")  # This line can be important in certain environments
    # Initialize the Chrome WebDriver with the specified options
    driver = webdriver.Chrome(options=options)

    driver = webdriver.Chrome()
    driver.get("https://www.bovada.lv/sports/football/nfl")

    # wait for the page to load
    time.sleep(10)
    driver.implicitly_wait(10)
    # get the HTML source
    html = driver.page_source

    # create a BeautifulSoup object
    soup = BeautifulSoup(html, "html.parser")

    # close the driver
    driver.quit()

    data = []
    sections = soup.find_all("section", {"class":"coupon-content more-info"})#soup.find_all("section", {"class":"coupon-content more-info"})
    for game in sections:
        try:
            item = str(game).split('>')
            info = [x.split('<')[0].strip() for x in item if not x.startswith("<")]
            data.append(info)
        except:
            pass

    df = pd.DataFrame(data)

    df["total_home"] = df.apply(lambda row: concat_values(row[16], row[17], row[18]), axis=1)
    df["total_away"] = df.apply(lambda row: concat_values(row[19], row[20], row[21]), axis=1)

    df.drop(columns = [3, 4, 5, 8, 9, 16, 17, 18, 19, 20, 21, 22], inplace=True)
    columns = ["date", "time", "bets", "home", "away", "spread_home_points", "spread_home_odds", 
               "spread_away_points", "spread_away_odds", "win_home", "win_away", "total_home", "total_away"]
    df.columns = columns

    #filtering to this week
    today = datetime.now()
    #remove plus from bets
    df['bets'] = df['bets'].apply(lambda x: x[2:])

    days_until_next_monday = (7 - today.weekday()) % 7
    if days_until_next_monday == 0:
        days_until_next_monday = 7  # If today is Monday, we take the next Monday
    next_monday = today + timedelta(days=days_until_next_monday)
    df["date"] = pd.to_datetime(df["date"], format="%m/%d/%y")
    df = df[df["date"] <= next_monday]
    
    df.reset_index(inplace=True, drop=True)

    # Applying the conversion to the 'win_home' and 'win_away' columns
    df['win_home'] = df['win_home'].apply(convert_to_int)
    df['win_away'] = df['win_away'].apply(convert_to_int)
    #ranking
    home = df[["home", "win_home"]].rename(columns={'home': 'team', 'win_home': 'odds'})
    away = df[['away', 'win_away']].rename(columns={'away': 'team', 'win_away': 'odds'})

    combined = pd.concat([home, away]).sort_values('odds', ascending=False)
    combined['index'] = combined.index
    combined.index = range(0, 2*len(combined), 2)
    df['points'] = None

    # Iterating over the combined DataFrame to assign ranks
    for i, x in combined.iterrows():
        df.at[x['index'], 'points'] = (i-len(combined))/2

    return df.sort_values('points', ascending=False)
    
df = get_data()

df.head()

,date,time,bets,home,away,spread_home_points,spread_home_odds,spread_away_points,spread_away_odds,win_home,win_away,total_home,total_away,points
7,2023-12-03,1:00 PM,196,Miami Dolphins,Washington Commanders,-9.5,(-115),+9.5,(-105),-480,350,O 49.5 (-110),U 49.5 (-110),12.0
0,2023-11-30,8:15 PM,258,Seattle Seahawks,Dallas Cowboys,+9.5,(-110),-9.5,(-110),340,-460,O 47.5 (-110),U 47.5 (-110),11.0
12,2023-12-04,8:15 PM,187,Cincinnati Bengals,Jacksonville Jaguars,+9.0,(-120),-9.0,(EVEN),320,-430,O 39.0 (-110),U 39.0 (-110),10.0
11,2023-12-03,8:20 PM,169,Kansas City Chiefs,Green Bay Packers,-6.0,(-105),+6.0,(-115),-250,210,O 42.5 (-110),U 42.5 (-110),9.0
1,2023-12-03,1:00 PM,155,Arizona Cardinals,Pittsburgh Steelers,+5.5,(-105),-5.5,(-115),210,-250,O 41.0 (-105),U 41.0 (-115),8.0


In [ ]:
import plotly.express as px

def generate_line_graph(historical_data, game_id_to_team):
    # Prepare data for plotting
    plot_data = []
    for game_id, data in historical_data.items():
        for point_type in ['Home Win', 'Away Win']:
            for point in data[point_type]:
                plot_data.append({'Game': game_id_to_team[game_id], 'Type': point_type, 'Value': point})

    df_plot = pd.DataFrame(plot_data)

    # Create the line graph
    fig = px.line(df_plot, x='Game', y='Value', color='Type', line_group='Game',
                  labels={'Value': 'Winning Points', 'Game': 'Game', 'Type': 'Team'})
    return fig

generate_line_graph('data_log.jsonl')